In [78]:
from generateTestData import shift, testDataGeneration
from datetime import date
import pandas as pd
import os
import numpy as np

filename = "csv/rides_web.csv"
df = pd.read_csv(filename, header=None, names=['shiftDate', 'shiftType','numRides','numRestHours'])
df['shiftDate'] = pd.to_datetime(df['shiftDate']) # convert object to datetime
df

,shiftDate,shiftType,numRides,numRestHours
0,2023-06-22,ND,7,3.0
1,2023-06-23,ND,8,1.0
2,2023-06-29,ND,2,9.0
3,2023-07-04,ND,1,10.0
4,2023-07-05,TD,12,3.0
5,2023-07-05,ND,1,9.0
6,2023-07-09,TD,4,7.0
7,2023-07-08,TD,4,6.0
8,2023-07-09,ND,5,3.5
9,2023-07-14,ND,1,10.0


# Remove Duplicates

In [79]:
# remove duplicates of data
df = df.drop_duplicates(
  subset = ['shiftDate', 'shiftType'],
  keep = 'last').reset_index(drop = True)
df

,shiftDate,shiftType,numRides,numRestHours
0,2023-06-22,ND,7,3.0
1,2023-06-23,ND,8,1.0
2,2023-06-29,ND,2,9.0
3,2023-07-04,ND,1,10.0
4,2023-07-05,TD,12,3.0
5,2023-07-05,ND,1,9.0
6,2023-07-09,ND,5,3.5
7,2023-07-14,ND,1,10.0
8,2023-07-17,TD,8,4.0
9,2023-07-17,ND,5,7.0


# sort by Shift-Date and Shift-Tpye

In [80]:
df = df.sort_values(['shiftDate', 'shiftType'], ascending=[False, False]).reset_index(drop=True)
df

,shiftDate,shiftType,numRides,numRestHours
0,2023-07-17,TD,8,4.0
1,2023-07-17,ND,5,7.0
2,2023-07-14,ND,1,10.0
3,2023-07-09,TD,4,7.0
4,2023-07-09,ND,5,3.5
5,2023-07-08,TD,4,6.0
6,2023-07-05,TD,12,3.0
7,2023-07-05,ND,1,9.0
8,2023-07-04,ND,1,10.0
9,2023-06-29,ND,2,9.0


# Merge columns with same date

In [81]:
s = df.groupby(['shiftDate']).cumcount()

df = df.set_index(['shiftDate', s]).unstack().sort_index(level=1, axis=1)
df.columns = [f'{x}{y}' for x, y in df.columns]
df = df.reset_index()
df

,shiftDate,numRestHours0,numRides0,shiftType0,numRestHours1,numRides1,shiftType1
0,2023-06-22,3.0,7.0,ND,NaN,NaN,NaN
1,2023-06-23,1.0,8.0,ND,NaN,NaN,NaN
2,2023-06-29,9.0,2.0,ND,NaN,NaN,NaN
3,2023-07-04,10.0,1.0,ND,NaN,NaN,NaN
4,2023-07-05,3.0,12.0,TD,9.0,1.0,ND
5,2023-07-08,6.0,4.0,TD,NaN,NaN,NaN
6,2023-07-09,7.0,4.0,TD,3.5,5.0,ND
7,2023-07-14,10.0,1.0,ND,NaN,NaN,NaN
8,2023-07-17,4.0,8.0,TD,7.0,5.0,ND


# Change rows if shiftType == 'ND'

In [82]:
s = df['shiftType0'] == 'ND' # boolean condition

#swap everything
df.loc[s, ['numRestHours0','numRestHours1']] = df.loc[s, ['numRestHours1','numRestHours0']].values
df.loc[s, ['numRides0','numRides1']] = df.loc[s, ['numRides1','numRides0']].values
df.loc[s, ['shiftType0','shiftType1']] = df.loc[s, ['shiftType1','shiftType0']].values

df

,shiftDate,numRestHours0,numRides0,shiftType0,numRestHours1,numRides1,shiftType1
0,2023-06-22,NaN,NaN,NaN,3.0,7.0,ND
1,2023-06-23,NaN,NaN,NaN,1.0,8.0,ND
2,2023-06-29,NaN,NaN,NaN,9.0,2.0,ND
3,2023-07-04,NaN,NaN,NaN,10.0,1.0,ND
4,2023-07-05,3.0,12.0,TD,9.0,1.0,ND
5,2023-07-08,6.0,4.0,TD,NaN,NaN,NaN
6,2023-07-09,7.0,4.0,TD,3.5,5.0,ND
7,2023-07-14,NaN,NaN,NaN,10.0,1.0,ND
8,2023-07-17,4.0,8.0,TD,7.0,5.0,ND


# latest date on top

In [83]:
df = df.sort_values('shiftDate', ascending=False).reset_index(drop=True)
df

,shiftDate,numRestHours0,numRides0,shiftType0,numRestHours1,numRides1,shiftType1
0,2023-07-17,4.0,8.0,TD,7.0,5.0,ND
1,2023-07-14,NaN,NaN,NaN,10.0,1.0,ND
2,2023-07-09,7.0,4.0,TD,3.5,5.0,ND
3,2023-07-08,6.0,4.0,TD,NaN,NaN,NaN
4,2023-07-05,3.0,12.0,TD,9.0,1.0,ND
5,2023-07-04,NaN,NaN,NaN,10.0,1.0,ND
6,2023-06-29,NaN,NaN,NaN,9.0,2.0,ND
7,2023-06-23,NaN,NaN,NaN,1.0,8.0,ND
8,2023-06-22,NaN,NaN,NaN,3.0,7.0,ND


# drop columns

In [84]:
df = df.drop(['shiftType0', 'shiftType1'], axis=1)
df

,shiftDate,numRestHours0,numRides0,numRestHours1,numRides1
0,2023-07-17,4.0,8.0,7.0,5.0
1,2023-07-14,NaN,NaN,10.0,1.0
2,2023-07-09,7.0,4.0,3.5,5.0
3,2023-07-08,6.0,4.0,NaN,NaN
4,2023-07-05,3.0,12.0,9.0,1.0
5,2023-07-04,NaN,NaN,10.0,1.0
6,2023-06-29,NaN,NaN,9.0,2.0
7,2023-06-23,NaN,NaN,1.0,8.0
8,2023-06-22,NaN,NaN,3.0,7.0


# format Date

In [85]:
df.insert(loc=0, column='Weekday', value=df["shiftDate"].dt.strftime('%a'))
df['shiftDate'] = df["shiftDate"].dt.strftime('%d.%m.%Y')
df = df[['Weekday','shiftDate', 'numRides0', 'numRestHours0', 'numRides1', 'numRestHours1']]
df

,Weekday,shiftDate,numRides0,numRestHours0,numRides1,numRestHours1
0,Mon,17.07.2023,8.0,4.0,5.0,7.0
1,Fri,14.07.2023,NaN,NaN,1.0,10.0
2,Sun,09.07.2023,4.0,7.0,5.0,3.5
3,Sat,08.07.2023,4.0,6.0,NaN,NaN
4,Wed,05.07.2023,12.0,3.0,1.0,9.0
5,Tue,04.07.2023,NaN,NaN,1.0,10.0
6,Thu,29.06.2023,NaN,NaN,2.0,9.0
7,Fri,23.06.2023,NaN,NaN,8.0,1.0
8,Thu,22.06.2023,NaN,NaN,7.0,3.0


In [86]:
x = "1.0"
y = x.replace(".0", "")
print(y)

1


In [87]:
df[['numRides0', 'numRides1']] = df[['numRides0', 'numRides1']].astype("Int64")
df = df.replace(np.nan, pd.NA)

# remove .0
df[['numRestHours0','numRestHours1']] = df[['numRestHours0','numRestHours1']].fillna(-1)
df[['numRestHours0', 'numRestHours1']] = df[['numRestHours0', 'numRestHours1']].astype(str)
df['numRestHours0'] = df['numRestHours0'].apply(lambda x: x.replace(".0",""))
df['numRestHours1'] = df['numRestHours1'].apply(lambda x: x.replace(".0",""))
df[['numRestHours0','numRestHours1']] = df[['numRestHours0','numRestHours1']].replace("-1", pd.NA)
df

,Weekday,shiftDate,numRides0,numRestHours0,numRides1,numRestHours1
0,Mon,17.07.2023,8,4,5,7
1,Fri,14.07.2023,<NA>,<NA>,1,10
2,Sun,09.07.2023,4,7,5,3.5
3,Sat,08.07.2023,4,6,<NA>,<NA>
4,Wed,05.07.2023,12,3,1,9
5,Tue,04.07.2023,<NA>,<NA>,1,10
6,Thu,29.06.2023,<NA>,<NA>,2,9
7,Fri,23.06.2023,<NA>,<NA>,8,1
8,Thu,22.06.2023,<NA>,<NA>,7,3


In [88]:
df.dtypes

Weekday          object
shiftDate        object
numRides0         Int64
numRestHours0    object
numRides1         Int64
numRestHours1    object
dtype: object

# reset index